In [24]:
#load path
import os
import sys
import astropy
from astropy.io import fits
from astropy.table import QTable

import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.stats import sigma_clipped_stats, sigma_clip

from astropy.wcs import WCS
from photutils.aperture import CircularAperture, aperture_photometry, CircularAnnulus, ApertureStats
from astropy.stats import SigmaClip

from astropy.visualization import (ImageNormalize, MinMaxInterval,PercentileInterval,SqrtStretch)
import glob

import warnings


In [25]:
path='./data/'
rawpath=r'/Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/'
calibpath='./data/Calibs/'
reduxpath='./data/Redux/'
combinepath='./data/Redux/Combined/'
gain=0.25 #e-/ADU at G = 125

In [26]:
def GAIA_query(center_ra, center_dec, radius, magmin=10, magmax=14, writedir='./calstars/'):
   from astroquery.gaia import GaiaClass
   from astropy.coordinates import SkyCoord
   import astropy.units as u


   coord = SkyCoord(ra=center_ra, dec=center_dec, unit=(u.deg, u.deg))
   gaiaobj = GaiaClass()
   gaiaobj.ROW_LIMIT = 50000  # High limit on number of rows returned
   job = gaiaobj.cone_search(coord, radius=radius * u.deg, verbose=False)
   results = job.get_results()

   selected = (results['has_xp_sampled']) & (results['phot_g_mean_mag'] < magmax) & (results['phot_g_mean_mag'] > magmin)
   starID = results['source_id'][selected]
   starRA = results['ra'][selected]
   starDEC = results['dec'][selected]

   print('Found {} stars'.format(len(starID)))
   ## Download the spectra:
   retrieval_type = 'XP_SAMPLED'
   datalink = gaiaobj.load_data(ids=starID,
                          data_release='Gaia DR3',
                          retrieval_type=retrieval_type,
                          data_structure='INDIVIDUAL',
                          format='votable',
                          verbose=False,
                          overwrite_output_file='True')

   dl_keys  = [inp for inp in datalink.keys()]

   strwave = []
   strflux = []

   #flux is in w/m2/nm, convert to erg/s/cm2/A
   conv_factor = 1e7 * 1e-4 * 1e-1 #1 W = 1e7 erg/s, 1 m2 = 1e4 cm2, 1 nm = 10 A

   for ind, strid in enumerate(starID):
         for dl_key in dl_keys:
              if str(strid) in dl_key:
                wave = datalink[dl_key][0].to_table().as_array()['wavelength']
                wave = np.ma.getdata(wave)*10
                flux = datalink[dl_key][0].to_table().as_array()['flux']
                flux = np.ma.getdata(flux)*conv_factor
                strwave.append(wave)
                strflux.append(flux)
         #Save in fits file
         hdu1 = fits.PrimaryHDU(flux)
         hdu1.header['SRCID'] = starID[ind]
         hdu1.header['RA'] = starRA[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['DEC'] = starDEC[ind]
         hdu1.header['FUNIT'] = 'erg/s/cm2/A'
         hdu1.header['CRVAL1'] = wave[0]
         hdu1.header['CD1_1'] = wave[1]-wave[0]
         hdu2 = fits.ImageHDU(wave)
         hdulist = fits.HDUList([hdu1, hdu2])
         hdulist.writeto(writedir+'GAIA3_{}.fits'.format(starID[ind]), overwrite=True)
    
   return starID, starRA, starDEC, np.array(strwave), np.array(strflux)

In [27]:
def make_star_library(header, img, caldir, force_query=False):

    #Find center of image in RA and DEC
    wcs = WCS(header)
    ny , nx = img.shape
    center_ra, center_dec = wcs.wcs_pix2world(nx/2, ny/2, 0)

    objname = header['OBJECT'].replace(' ','_')
    caltargdir = caldir+objname+'/'
    if not os.path.exists(caltargdir):
       os.makedirs(caltargdir)

    fcalstr = glob.glob(caltargdir+'GAIA3_*.fits')
    if len(fcalstr) == 0 or force_query==True:

      print('  Running GAIA query....')
      #Query GAIA for stars in the field
      radius = 0.4 #degrees
      strID, strRA, strDEC, specwave, specflux = GAIA_query(center_ra, center_dec, radius, \
                                                 magmin=10, magmax=14, writedir=caltargdir)
    else:
      print('  Reading GAIA spectra from: {}'.format(caltargdir))
      strID = []
      strRA = []
      strDEC = []
      specwave = []
      specflux = []

      for fcal in fcalstr:
          hdu = fits.open(fcal)
          strID.append(hdu[0].header['SRCID'])
          strRA.append(hdu[0].header['RA'])
          strDEC.append(hdu[0].header['DEC'])
          specwave.append(hdu[1].data)
          specflux.append(hdu[0].data)
          hdu.close()

      specwave = np.array(specwave)
      specflux = np.array(specflux) 
      
    #For each star, find its pixel position in the image
    star_positions = []
    for ra, dec in zip(strRA, strDEC):
        x, y = wcs.wcs_world2pix(ra, dec, 0)
        star_positions.append((x, y))
    star_positions = np.array(star_positions)

    #keep only stars within the image boundaries
    okstars =  (0 < star_positions[:,0]) & (star_positions[:,0] < nx) \
               & (star_positions[:,1] < ny) & (0 < star_positions[:,1])        
    
    star_positions = star_positions[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]

    return star_positions, specwave, specflux


In [91]:
def flux_cal_gaia(img, varima, header, filter, caldir='./calstars/'):

    star_positions, specwave, specflux = make_star_library(header, img, caldir)
    if len(star_positions)==0:
       star_positions, specwave, specflux = make_star_library(header, img, caldir, force_query=True)
       if len(star_positions)==0:
           return 0, 0, 0, 2 #Mask value = 2 means no stars found even after forcing query

    #Perform aperture photometry on each star with background subtraction
    apertures = CircularAperture(star_positions, r=10)
    annulus = CircularAnnulus(star_positions, r_in=12, r_out=15)

    sigclip = SigmaClip(sigma=3.0, maxiters=10)
    aper_stats = ApertureStats(img, apertures, sigma_clip=None)
    bkg_stats = ApertureStats(img, annulus, sigma_clip=sigclip)
    bkg_sum = bkg_stats.median * aper_stats.sum_aper_area.value
    inst_flux = aper_stats.sum - bkg_sum

    mask = np.isnan(varima).astype(int)

    mask_table = aperture_photometry(mask,apertures)
    satpix = mask_table['aperture_sum']

    #Keep only stars with positive fluxes and unsaturated
    okstars = (inst_flux>0) & (np.isfinite(inst_flux)) & (satpix==0)
    if okstars.sum()==0:
        print('No valid stars found for flux calibration. All stars are either saturated or have non-positive fluxes.')
        return [0, 0, 0, 0, 4], [0, 0, 0, 0] #Mask value = 4 means no valid stars left after removing saturation and negative fluxes

    star_positions = star_positions[okstars]
    inst_flux = inst_flux[okstars]
    specwave = specwave[okstars,:]
    specflux = specflux[okstars,:]

    #Read appropriate filter file
    filterdict = {'R':'r', 'G':'g', 'I':'i', 'SII':'SII', 'OIII':'OIII', 'Ha':'Ha', 'Hb':'Hb'}
    filterfile = './filters/Tobi_'+filterdict[filter]+'.txt'
    filterdata = np.loadtxt(filterfile)

    #compute filter pivot wavelength
    filter_wave = filterdata[:,1]
    filter_trans = filterdata[:,2]
    num = np.trapz(filter_wave * filter_trans, filter_wave)
    denom = np.trapz(filter_trans, filter_wave)
    filter_pivot = num / denom

    #convolve each spectrum with the filter
    calfactors = []
    psffwhm = []
    psfaxratio = []

    for i in range(okstars.sum()):
        wave = specwave[i]
        flux = specflux[i]
        #interpolate filter to spectrum wavelength
        filter_interp = np.interp(wave, filter_wave, filter_trans, left=0, right=0)
        #compute the integral of flux * filter
        num = np.trapz(flux * filter_interp, wave)
        denom = np.trapz(filter_interp, wave)
        flux_cal = num / denom  # erg/s/cm2/A
        #compute the calibration factor
        calfactor = flux_cal / inst_flux[i]  # erg/s/cm2/A per e-/s
        calfactors.append(calfactor)

    #return median calibration factor
    calfactors=np.array(calfactors)
    mean_calfactor, median_calfactor, std_calfactor = sigma_clipped_stats(calfactors, sigma=3.5)
    #Compute AB ZP from this calib factor
    median_calfacnu = median_calfactor * (filter_pivot**2) / 2.99792458e18
    median_abzp = -2.5 * np.log10(median_calfacnu) - 48.6

    #Done with fluxcal now do PSF fitting
    from photutils.psf import PSFPhotometry, GaussianPRF
    from photutils.background import LocalBackground, MMMBackground
    localbkg_estimator = LocalBackground(8, 14, MMMBackground())

    psf_model = GaussianPRF()
    psf_model.x_0.fixed=True
    psf_model.y_0.fixed=True
    psf_model.x_fwhm.fixed = False
    psf_model.y_fwhm.fixed = False
    psf_model.theta.fixed = False
    init_params = QTable()
    init_params['x'] = star_positions[:,0]
    init_params['y'] = star_positions[:,1]
    init_params['x_fwhm'] = np.zeros_like(init_params['x'])+4
    init_params['y_fwhm'] = np.zeros_like(init_params['y'])+4

    #Generate error image and ensure all values are positive
    std = np.nan_to_num(np.sqrt(varima))
    std[std==0] = 1e6

    #print(std[int(star_positions[i,1]-6):int(star_positions[i,1]+7), int(star_positions[i,0]-6):int(star_positions[i,0]+7)])
    #print(img[int(star_positions[i,1]-6):int(star_positions[i,1]+7), int(star_positions[i,0]-6):int(star_positions[i,0]+7)])
    psfphot = PSFPhotometry(psf_model, (9,9), aperture_radius=4, localbkg_estimator=localbkg_estimator)
    phot = psfphot(img, error=std, init_params=init_params)
    #phot.show_in_browser(jsviewer=True)

    for i in range(okstars.sum()):
     if phot['flux_fit'][i]>0:
       psffwhm.append(np.min([phot['x_fwhm_fit'][i],phot['y_fwhm_fit'][i]]))
       psfaxratio.append(np.max([phot['x_fwhm_fit'][i],phot['y_fwhm_fit'][i]])/np.min([phot['x_fwhm_fit'][i],phot['y_fwhm_fit'][i]]))

    print('  Image Calibration factor in filter {}: {:.2e} +/- {:.2e} (AB ZP {:.2f})'.format(filter, median_calfactor, std_calfactor, median_abzp))

    psffwhm=np.array(psffwhm)
    mean_fwhm, median_fwhm, std_fwhm = sigma_clipped_stats(psffwhm, sigma=3.5)

    psfaxratio=np.array(psfaxratio)
    mean_axratio, median_axratio, std_axratio = sigma_clipped_stats(psfaxratio, sigma=3.5)

    print('  PSF FWHM: {:.2f} arcsec. Axis Ratio: {:.2f}'.format(median_fwhm*0.55, median_axratio))


    return [median_calfactor, std_calfactor, median_abzp, len(calfactors), 0], [median_fwhm, std_fwhm, median_axratio, std_axratio]



In [92]:
#now reduce science images
#in every directory of rawpath, find if there is a LIGHT folder and reduce the images inside

diffrun=True
addsuperflat=False
skipfcal = False

cutsize = 400
for root, dirs, files in os.walk(rawpath):
    if 'LIGHT' in dirs:

        light_dir = os.path.join(root, 'LIGHT')
        sciencelist=glob.glob(light_dir+'/*sci*.fits')
        #sciencelist=glob.glob(light_dir+'/2025-11-07*sci_PSZ*.fits')
        if len(sciencelist)==0:
            continue

        date = root.split('/')[-1]

        if date == '2025-11-16':
          continue

        print('Reducing science images for date: {}'.format(date))
        #find closest date in calibpath
        allbias = glob.glob(calibpath+'/MasterBias_*.fits')
        alldates = [os.path.basename(f).split('_')[1].split('.')[0] for f in allbias]
        date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
        closest_date = alldates[np.argmin(date_diffs)]
        print('Using calibration bias from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

        masterbias=fits.open(calibpath+'/MasterBias_{}.fits'.format(closest_date))[0].data

        for img in sciencelist:
          if 'Orion' in img:
              continue
          if diffrun and os.path.isfile(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1]):
              print('  Skipping file {}'.format(img))
              continue
          else:
           print('  Running file {}'.format(img))
           hdu=fits.open(img)
           data=hdu[0].data
           header=hdu[0].header
           filter=header['FILTER']
           exptime=header['EXPTIME']
           binning=header['XBINNING']*header['YBINNING']
           #Read also flat and dark (darks are fixed in time so far)

           allflats = glob.glob(calibpath+'/MasterFlat_{}_*.fits'.format(filter))
           alldates = [os.path.basename(f).split('_')[-1].split('.')[0] for f in allflats]
           date_diffs = [abs((astropy.time.Time(date) - astropy.time.Time('{}-{}-{}'.format(d[:4],d[4:6],d[6:8]))).jd) for d in alldates]
           closest_date = alldates[np.argmin(date_diffs)]
           print('  Using calibration flat from date: {}-{}-{}'.format(closest_date[:4],closest_date[4:6],closest_date[6:8]))

           normflat=fits.open(calibpath+'MasterFlat_{}_{}.fits'.format(filter, closest_date))[0].data
           try:
               masterdark = fits.open(calibpath+'MasterDark_'+str(int(exptime))+'s.fits')[0].data
               masterdark -= masterbias
               darkval = np.median(masterdark)
           except:
               masterdark = fits.open(calibpath+'MasterDark_1200s.fits')[0].data
               masterdark -= masterbias
               darkval = np.median(masterdark)/1200.*exptime
               print('  No matching dark found for exptime = {}s, using scaled 1200s dark'.format(exptime))
           try:
               if addsuperflat:
                     dummy = fits.open(calibpath+'SuperFlat_{}.fits'.format(filter))[0].data
                     superflat= np.ones_like(normflat)
                     superflat[cutsize:-cutsize,cutsize:-cutsize] = dummy
                     normflat = normflat * superflat
           except:
                print('  No superflat found for filter {}, proceeding without it'.format(filter))

           raw_elec=binning*gain*(data-masterbias)#-darkval
           reduced=raw_elec/normflat/exptime
           vardata=raw_elec/normflat**2/exptime**2

           #Make a saturation Mask
           satmask = (data>60000)

           vardata[satmask] = np.nan

           #Crop cutsize pixels from each edge to avoid vignetting, preserve WCS
           reduced = reduced[cutsize:-cutsize,cutsize:-cutsize]
           satmask = satmask[cutsize:-cutsize,cutsize:-cutsize]
           vardata = vardata[cutsize:-cutsize,cutsize:-cutsize]
           try:
              header['CRPIX1'] -= cutsize
              header['CRPIX2'] -= cutsize
           except:
              #raise a warning
              warnings.warn('No WCS found in header, skipping CRPIX update')

           #Now run fluxcal
           if not skipfcal:
               calpars, psfpars = flux_cal_gaia(reduced, vardata, header, filter, caldir='./calstars/')
               if np.logical_not(np.isfinite(calpars[0])):
                   calpars = [0,0,0,3]
               if calpars[0] !=0:
                  reduced *= calpars[0]/1e-17
                  vardata *= (calpars[0]/1e-17)**2

               header['FCALFAC'] = calpars[0]
               header['FCALSTD'] = calpars[1]
               header['FCALSTR'] = calpars[3]
               header['ABZP'] = calpars[2]
               header['FUNIT'] = '1e-17 erg/s/cm2/A'
               header['PSFFWHM'] = psfpars[0]
               header['PSFAXRAT'] = psfpars[2]
           else:
               header['FCALFAC'] = 0
               header['FCALSTD'] = 0
               header['FCALSTR'] = 0
               header['ABZP'] = 0
               header['FUNIT'] = 'e-/s'
               header['PSFFWHM'] = 0
               header['PSFAXRAT'] = 0

           #Once all the processing is done, compute mean, median and std of image
           mean, median, std = sigma_clipped_stats(reduced, sigma=3.5)
           header['IMGMEAN']  = mean
           header['IMGMED']   = median
           header['IMGSTD']   = std

           if addsuperflat:
              fits.writeto(reduxpath+"/"+img.replace('.fits','_reduced2.fits').split('/')[-1],reduced,header,overwrite=True)
           else:
              hdu1 = fits.PrimaryHDU(reduced.astype(np.float32), header)
              hdu2 = fits.ImageHDU(vardata.astype(np.float32), header)
              hduout = fits.HDUList([hdu1, hdu2])
              hduout.writeto(reduxpath+"/"+img.replace('.fits','_reduced.fits').split('/')[-1],overwrite=True)


Reducing science images for date: 2025-09-28
Using calibration bias from date: 2025-09-25
  Running file /Users/matteo/Google Drive/My Drive/TelescopeData/NEW CAMERA/2025-09-28/LIGHT/2025-09-28_20-22-44_sci_BD25_SII_exp120.00_0001.fits
  Using calibration flat from date: 2025-09-30


Set MJD-AVG to 60946.849824 from DATE-AVG'. [astropy.wcs.wcs]
/var/folders/ls/6hrs6lfd0h3_n2gkfnmq_k940000gn/T/ipykernel_18990/2063436078.py:90: RuntimeWarning: invalid value encountered in sqrt
  std = np.nan_to_num(np.sqrt(varima))


  Reading GAIA spectra from: ./calstars/BD25/
  Image Calibration factor in filter SII: 4.43e-16 +/- 2.98e-17 (AB ZP 16.84)
  PSF FWHM: 2.66 arcsec. Axis Ratio: 1.38


In [ ]:
'''
#Generate superflats for each filter, does not really work

#Find observations in each filter
filters = ['R','G','I','SII','OIII','Ha','Hb']

for filter in filters:
    reduximages = glob.glob(reduxpath+'/*_{}_*_reduced.fits'.format(filter))
    if len(reduximages) < 10:
        print('Not enough images found for filter {}'.format(filter))
        continue
    else:
        print('Found {} images for filter {}'.format(len(reduximages), filter))

    #Stack images to create superflat
    stack = []
    for filein in reduximages:
        hdu = fits.open(filein)
        exptime = hdu[0].header['EXPTIME']
        if exptime<200:
            continue
        data = (hdu[0].data).astype(np.float32)
        bkg = sep.Background(data)
        data_sub = data - bkg
        sources, segmap = sep.extract(data_sub, 0.5, minarea=10, err=bkg.globalrms, segmentation_map=True)
        data[segmap>0] = np.nan
        print(filein, np.shape(data))
        stack.append(data/np.nanmedian(data))

    stack = np.array(stack)
    #Compute median
    superflat = np.nanmedian(stack, axis=0)
    #Normalize superflat
    superflat /= np.nanmedian(superflat)
    #Save superflat
    fits.writeto(calibpath+'/SuperFlat_{}.fits'.format(filter), superflat, overwrite=True)
'''

In [ ]:
'''#for each image, run sep, mask sources and fit sky background
import sep
import numpy as np

reduximages = glob.glob(reduxpath+'/*_reduced.fits')
diffrun=True
#print(reduximages)

for fileout in reduximages:
   if diffrun and os.path.isfile(fileout.replace('_reduced.fits','_sky_sub.fits')):
    print('Skipping file {}'.format(fileout))
    continue
   else:
    hdu = fits.open(fileout)
    data = (hdu[0].data).astype(np.float32)
    bkg = sep.Background(data)

    data_sub = data - bkg
    sources, segmap = sep.extract(data_sub, 0.5, minarea=10, err=bkg.globalrms, segmentation_map=True)
    segmap[segmap>0] = 1

    bkg = sep.Background(data, bh=128, bw=128,mask=segmap)

    sky_median = np.median(data[segmap==0])
    maskravel = (segmap==0).ravel()
    #fit a 2D polynomial to the sky background using astropy
    y, x = np.mgrid[:data.shape[0], :data.shape[1]]
    A = np.c_[x.ravel()**2, y.ravel()**2, x.ravel()*y.ravel(), x.ravel(), y.ravel(), np.ones(data.shape).ravel()]
    C, _, _, _ = np.linalg.lstsq(A[maskravel], data.ravel()[maskravel], rcond=None)
    sky_fit = (C[0]*x**2 + C[1]*y**2 + C[2]*x*y + C[3]*x + C[4]*y + C[5]).reshape(data.shape)
    data_sky_sub = data - sky_fit

    hdu[0].data = data_sky_sub
    hdu.writeto(fileout.replace('_reduced.fits','_sky_sub.fits'), overwrite=True)
'''

In [93]:
#for each image, run sep, mask sources and fit sky background
import sep
import numpy as np

reduximages = glob.glob(reduxpath+'/*_reduced.fits')
diffrun=True

filttarg = {'M103':4, 'M29':4, 'M39':4, 'PSZ2G114.79-33.71':4, 'NGC4569':12, 'NGC7331':9}
#print(reduximages)

for fileout in reduximages:
   if diffrun and os.path.isfile(fileout.replace('_reduced.fits','_sky_sub.fits')):
    print('Skipping file {}'.format(fileout))
    continue
   else:
     with warnings.catch_warnings():
        warnings.filterwarnings('ignore')
        hdu = fits.open(fileout)
        data = (hdu[0].data).astype(np.float32)
        bkg = sep.Background(data)
        wcs = WCS(hdu[0].header)

        targname = os.path.basename(fileout).split('_')[3]
        #query astroquery to get the size of the target in arcmin
        from astroquery.simbad import Simbad

        result_table = Simbad.query_object(targname)

        if result_table is None:
            print('Could not find target {} in Simbad, using default size of 10 arcmin'.format(targname))
            xpos = data.shape[1] / 2
            ypos = data.shape[0] / 2
        else:
            try:
             #Find object RA and DEC
             obj_ra = result_table['ra'][0]
             obj_dec = result_table['dec'][0]
             print('Query succeeded')
             from astropy.coordinates import SkyCoord
             from astropy import units as u
             xpos, ypos = wcs.wcs_world2pix(obj_ra, obj_dec, 0)
            except:
             xpos = data.shape[1] / 2
             ypos = data.shape[0] / 2

        #Convert size from arcmin to pixels
        size_arcmin = 10
        size_pix = (size_arcmin * 60.) / 0.55 / 2

        data_sub = data - sep.Background(data, fw=filtsize, fh=filtsize)
        try:
            sources, segmap = sep.extract(data_sub, 1.0, minarea=10, err=bkg.globalrms, segmentation_map=True)
        except:
            sources, segmap = sep.extract(data_sub, 2.5, minarea=10, err=bkg.globalrms, segmentation_map=True, deblend_cont=1.0)
        segmap[segmap>0] = 1

        #Mask a circular region around the target only if targname is not M29 or M39 or M103
        if targname != 'M29' and targname != 'M39' and targname != 'M103' and targname!='PSZ2G114.79-33.71':
           y, x = np.mgrid[:data.shape[0], :data.shape[1]]
           r = np.sqrt((x - xpos)**2 + (y - ypos)**2)
           segmap[r < size_pix] = 1

        try:
            filtsize = filttarg[targname]
        except:
            filtsize = 9

        bkg = sep.Background(data, fw=filtsize, fh=filtsize, mask=segmap)

        hdu[0].data = data - bkg
        hdu.writeto(fileout.replace('_reduced.fits','_sky_sub.fits'), overwrite=True)


Skipping file ./data/Redux/2025-09-28_20-43-50_sci_M39_R_exp120.00_0001_reduced.fits
Skipping file ./data/Redux/2025-11-28_19-15-52_sci_M103_G_exp300.00_0000_reduced.fits
Skipping file ./data/Redux/2025-10-07_21-06-04_sci_M103_I_exp300.00_0002_reduced.fits
Skipping file ./data/Redux/2025-10-10_18-22-10_sci_M103_R_exp300.00_0001_reduced.fits
Skipping file ./data/Redux/2025-11-03_19-45-24_sci_NGC6946_Ha_exp1200.00_0003_reduced.fits
Skipping file ./data/Redux/2025-09-28_20-45-51_sci_M39_R_exp120.00_0002_reduced.fits
Skipping file ./data/Redux/2025-09-28_19-39-14_sci_BD28_Hb_exp120.00_0001_reduced.fits
Skipping file ./data/Redux/2025-11-27_20-21-46_sci_PSZ2G114.79-33.71_G_exp300.00_0011_reduced.fits
Skipping file ./data/Redux/2025-10-16_20-29-19_sci_M29_I_exp120.00_0002_reduced.fits
Skipping file ./data/Redux/2025-11-11_18-50-42_sci_PSZ2G114.79-33.71_R_exp300.00_0003_reduced.fits
Skipping file ./data/Redux/2025-09-29_19-16-35_sci_NGC6946_I_exp600.00_0002_reduced.fits
Skipping file ./data/R